In [2]:
:dep plotters = { version = "^0.3.6", default-features = false, features = ["evcxr", "all_series", "all_elements"] }
extern crate plotters;
use plotters::prelude::*;

fn draw_chart(data: &Vec<(f32, f32)>, name: impl ToString) -> plotters::evcxr::SVGWrapper {
    let minx = data.iter().min_by(|a, b| a.0.partial_cmp(&b.0).unwrap_or(std::cmp::Ordering::Equal)).unwrap().0;
    let maxx = data.iter().max_by(|a, b| a.0.partial_cmp(&b.0).unwrap_or(std::cmp::Ordering::Equal)).unwrap().0;
    let miny = data.iter().min_by(|a, b| a.1.partial_cmp(&b.1).unwrap_or(std::cmp::Ordering::Equal)).unwrap().1;
    let maxy = data.iter().max_by(|a, b| a.1.partial_cmp(&b.1).unwrap_or(std::cmp::Ordering::Equal)).unwrap().1;
    let figure = evcxr_figure((640, 480), |root| {
        root.fill(&WHITE)?;
        let mut chart = ChartBuilder::on(&root)
            .caption(name.to_string(), ("Arial", 50).into_font())
            .margin(5)
            .x_label_area_size(30)
            .y_label_area_size(30)
            .build_cartesian_2d(minx..maxx, miny..maxy)?;

        chart.configure_mesh().draw()?;

        chart.draw_series(LineSeries::new(
            data.clone(),
            &RED,
        )).unwrap();

        // chart.configure_series_labels()
        //     .background_style(&WHITE.mix(0.8))
        //     .border_style(&BLACK)
        //     .draw()?;
        Ok(())
    });
    return figure;
}

In [3]:
:dep num = { version = "^0.4.3" }

use num::BigRational as R;
use num::BigInt as I;
use num::BigUint as U;
use num::Integer;
use num::traits::ConstZero;
use num::FromPrimitive;
use num::ToPrimitive;

use std::ops::AddAssign;

fn factorial(n: &U) -> R {
    let mut c = n.clone();
    let one = I::from_i8(1).unwrap();
    let mut out = R::new(one.clone(), one.clone());
    while c > U::ZERO {
        out *= R::new(I::from_biguint(num::bigint::Sign::Plus, c.clone()), one.clone());
        c -= 1u32;
    }
    out
}

fn whatever(alpha1: &R, alpha2: &R, n1: usize, n2: usize) -> R {
    (alpha1.pow(n1 as i32) / factorial(&u(n1))) *
    (alpha2.pow(n2 as i32) / factorial(&u(n2))) *
    factorial(&u(n1+n2))
}

fn stationary_distribution_at_zero(n1_max: usize, n2_max: usize, alpha1: R, alpha2: R) -> R {
    let mut sum = R::from_float(0.0).unwrap();
    for n1 in 0..=n1_max {
        for n2 in 0..=n2_max {
            // println!("{n1} {n2}");
            sum += whatever(&alpha1, &alpha2, n1, n2);
        }
    }

    R::from_float(1.0).unwrap()/sum
}
fn u(i: usize) -> U {
    U::from_usize(i).unwrap()
}

fn rr(i: f64) -> R {
    R::from_float(i).unwrap()
}

fn stationary_distribution_at(zero: R, n1: usize, n2: usize, alpha1: R, alpha2: R) -> R {
    zero * whatever(&alpha1, &alpha2, n1, n2)
}

In [4]:
let lambda1 = 8.;
let lambda2 = 6.;
let theta1 = 2.;
let theta2 = 3.;
let c = 10;
let n1_max = 20;
let n2_max = 30;

let rho1 = lambda1 * theta1;
let rho2 = lambda2 * theta2;
let a1 = rho1 / c as f64;
let a2 = rho2 / c as f64;

let zero = stationary_distribution_at_zero(n1_max, n2_max, rr(a1), rr(a2));
zero.to_f64()

Some(1.9345027205245324e-26)

In [5]:
let mut v = vec![];
for n1 in 0..=n1_max {
    let mut r = vec![];
    for n2 in 0..=n2_max {
        r.push(stationary_distribution_at(zero.clone(), n1, n2, rr(a1), rr(a2)));
    }
    v.push(r)
}


()

In [6]:
let sum: R = v.iter().map(|v| v.iter().sum::<R>()).sum();
println!("sum should be 1: {sum}");

sum should be 1: 1


In [7]:
let N1 = lambda1 * (theta1) / (theta1 * lambda1 + theta2 * lambda2);
let N2 = lambda2 * (theta2) / (theta1 * lambda1 + theta2 * lambda2);
println!("average number of requests in flight: {N1}, {N2}");

average number of requests in flight: 0.47058823529411764, 0.5294117647058824


In [8]:
let T1 = N1 / lambda1;
let T2 = N2 / lambda2;
println!("Avg service time: {T1}, {T2}");

Avg service time: 0.058823529411764705, 0.08823529411764706


In [26]:
let mut avg_service_time = vec![];
let mut avg_service_requests = vec![];
for lambda1 in 1..=100 {
    let lambda1 = lambda1 as f64;
    let n1 = lambda1 * (theta1) / (theta1 * lambda1 + theta2 * lambda2);
    avg_service_requests.push((lambda1 as f32, n1 as f32));
    avg_service_time.push((lambda1 as f32, (n1/lambda1) as f32));
}
avg_service_requests

[(1.0, 0.1), (2.0, 0.18181819), (3.0, 0.25), (4.0, 0.30769232), (5.0, 0.35714287), (6.0, 0.4), (7.0, 0.4375), (8.0, 0.47058824), (9.0, 0.5), (10.0, 0.5263158), (11.0, 0.55), (12.0, 0.5714286), (13.0, 0.59090906), (14.0, 0.6086956), (15.0, 0.625), (16.0, 0.64), (17.0, 0.65384614), (18.0, 0.6666667), (19.0, 0.6785714), (20.0, 0.6896552), (21.0, 0.7), (22.0, 0.7096774), (23.0, 0.71875), (24.0, 0.72727275), (25.0, 0.7352941), (26.0, 0.74285716), (27.0, 0.75), (28.0, 0.7567568), (29.0, 0.7631579), (30.0, 0.7692308), (31.0, 0.775), (32.0, 0.7804878), (33.0, 0.78571427), (34.0, 0.7906977), (35.0, 0.79545456), (36.0, 0.8), (37.0, 0.8043478), (38.0, 0.80851066), (39.0, 0.8125), (40.0, 0.81632656), (41.0, 0.82), (42.0, 0.8235294), (43.0, 0.8269231), (44.0, 0.8301887), (45.0, 0.8333333), (46.0, 0.8363636), (47.0, 0.83928573), (48.0, 0.84210527), (49.0, 0.8448276), (50.0, 0.84745765), (51.0, 0.85), (52.0, 0.852459), (53.0, 0.8548387), (54.0, 0.85714287), (55.0, 0.859375), (56.0, 0.86153847), (57.0

In [27]:
draw_chart(&avg_service_requests, "lambda1 vs N_1")

lambda1 vs N_1
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
 
10.0
 
 
 
20.0
 
 
 
30.0
 
 
 
40.0
 
 
 
50.0
 
 
 
60.0
 
 
 
70.0
 
 
 
80.0
 
 
 
90.0
 
 
 
100.0
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="35,444 41,406 47,373 53,346 59,322 65,302 71,284 77,269 83,255 89,242 95,231 101,221 107,211 113,203 119,195 125,188 131,182 137,176 143,170 149,165 156,160 162,155 168,151 174,147 180,143 186,139 192,136 198,133 204,130 210,127 216,124 222,122 228,119 234,117 240,114 246,112 252,110 258,108 264,106 270,104 277,103 283,101 289,99 295,98 301,96 307,95 313,94 319,92 325,91 331,90 337,89 343,87 349,86 355,85 361,84 367,83 373,82 379,81 385,80 391,79 398,78 404,77 410,77 416,76 422,75 428,74 434,74 440,73 446,72 452,71 458,71 464,70 470,69 476,69 482,68 488,68 494,67 500,66 506,66 512,65 519,65 525,64 531,64 537,63 543,63 549,62 555,62 561,61 567,61 573,60 579,60 585,60 591,59 597,59 603,58 609,58 615,58 621,57 627,57 634,56 "/>

In [28]:
avg_service_time

[(1.0, 0.1), (2.0, 0.09090909), (3.0, 0.083333336), (4.0, 0.07692308), (5.0, 0.071428575), (6.0, 0.06666667), (7.0, 0.0625), (8.0, 0.05882353), (9.0, 0.055555556), (10.0, 0.05263158), (11.0, 0.05), (12.0, 0.04761905), (13.0, 0.045454547), (14.0, 0.04347826), (15.0, 0.041666668), (16.0, 0.04), (17.0, 0.03846154), (18.0, 0.037037037), (19.0, 0.035714287), (20.0, 0.03448276), (21.0, 0.033333335), (22.0, 0.032258064), (23.0, 0.03125), (24.0, 0.030303031), (25.0, 0.029411765), (26.0, 0.028571429), (27.0, 0.027777778), (28.0, 0.027027028), (29.0, 0.02631579), (30.0, 0.025641026), (31.0, 0.025), (32.0, 0.024390243), (33.0, 0.023809524), (34.0, 0.023255814), (35.0, 0.022727273), (36.0, 0.022222223), (37.0, 0.02173913), (38.0, 0.021276595), (39.0, 0.020833334), (40.0, 0.020408163), (41.0, 0.02), (42.0, 0.019607844), (43.0, 0.01923077), (44.0, 0.018867925), (45.0, 0.018518519), (46.0, 0.018181818), (47.0, 0.017857144), (48.0, 0.01754386), (49.0, 0.01724138), (50.0, 0.016949153), (51.0, 0.0166666

In [29]:
draw_chart(&avg_service_time, "lambda1 vs T_1")

lambda1 vs T_1
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.01
 
 
 
0.02
 
 
 
0.03
 
 
 
0.04
 
 
 
0.05
 
 
 
0.06
 
 
 
0.07
 
 
 
0.08
 
 
 
0.09
 
 
 
0.1
 
 
 
 
10.0
 
 
 
20.0
 
 
 
30.0
 
 
 
40.0
 
 
 
50.0
 
 
 
60.0
 
 
 
70.0
 
 
 
80.0
 
 
 
90.0
 
 
 
100.0
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="35,56 41,95 47,128 53,155 59,179 65,199 71,217 77,232 83,246 89,259 95,270 101,280 107,290 113,298 119,306 125,313 131,319 137,325 143,331 149,336 156,341 162,346 168,350 174,354 180,358 186,362 192,365 198,368 204,371 210,374 216,377 222,379 228,382 234,384 240,387 246,389 252,391 258,393 264,395 270,397 277,398 283,400 289,402 295,403 301,405 307,406 313,407 319,409 325,410 331,411 337,412 343,414 349,415 355,416 361,417 367,418 373,419 379,420 385,421 391,422 398,423 404,424 410,424 416,425 422,426 428,427 434,427 440,428 446,429 452,430 458,430 464,431 470,432 476,432 482,433 488,433 494,434 500,435 506,435 512,436 519,436 525,437 531,437 537,438 543,438 549,439 555,439 561,440 567,440 573,441 579,441 585,441 591,442 597,442 603,443 609,443 615,443 621,444 627,444 634,444 "/>